In [1]:
import sys
import os
sys.path.append(os.getenv("PYTHONPATH", "/app")) #REVIEW
from utils import Utils

In [2]:
utils = Utils()
spark = utils.get_spark_session()
emp = utils.get_employee_data()
dptm = utils.get_department_data()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/09 21:12:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
dptm.show()

In [ ]:
# It is possible to change the number of partitions in a DataFrame in spark by utilizing repartition() or coalesce()
# repartition redistributes the data evenly across partitions. Useful when increasing the number of partitions or balancing data:
# This is useful when preparing for parallel computation
print(emp.rdd.getNumPartitions())
less_partitions_emp = emp.repartition(4)
more_partitions_emp = emp.repartition(50)

In [ ]:
print(less_partitions_emp.rdd.getNumPartitions())
print(more_partitions_emp.rdd.getNumPartitions())

In [ ]:
# Coalesce only reduces the number of partitions. But it avoids full shuffling by merging partitions instead of redistributing:
# This is useful when writting writing a final output
repartitioned_emp = emp.coalesce(2) # pyspark has a coalesce() function to be called on columns, which is different from this coalesce to be called on dataframes
print(repartitioned_emp.rdd.getNumPartitions())

In [ ]:
# The repartition can be done across specified columns. Repartitioning the data into 4 partitions by 'department_id' column.:
emp.repartition(4, "department_id").rdd.getNumPartitions() # N columns can be specified to partition by

In [ ]:
# It is possible to visualize the data partitioning with spark_partition_id:
from pyspark.sql.functions import spark_partition_id

emp.repartition(4, "department_id").withColumn("partition_id", spark_partition_id()).show() # Here every department_id is in a single partition

In [ ]:
# Join operations can be done utiling the join function:
emp_join = emp.join(dptm, how="inner", on=emp.department_id==dptm.department_id)
emp_join.show()

In [ ]:
# To select the fields, aliasing can be used since there are ambiguous fields:
emp_join.select(emp.name, dptm.department_name, dptm.department_id, emp.salary).show()

In [ ]:
# Or the alias can be explicitly defined:
emp.alias("e").join(dptm.alias("d"), how="inner", on=emp.department_id==dptm.department_id).select("e.name", "d.department_name", "d.department_id", "e.salary").show() # The quotes are
# necessary because 'e.name' and the remaining are column names now

INNER JOIN: Matches only existing keys in both DataFrames:

df.join(df2, on="id", how="inner").show()

LEFT JOIN: All left DataFrame rows, matched ones from right:

df.join(df2, on="id", how="left").show()

RIGHT JOIN: All right DataFrame rows, matched ones from left:

df.join(df2, on="id", how="right").show()

FULL OUTER JOIN: All rows from both DataFrames:

df.join(df2, on="id", how="full").show()

LEFT SEMI JOIN: Returns only left rows where a match exists:

df.join(df2, on="id", how="left_semi").show()

LEFT ANTI JOIN: Returns only left rows where no match exists:

df.join(df2, on="id", how="left_anti").show()

CROSS JOIN: Cartesian product of both tables:

df.crossJoin(df2).show()

In [ ]:
# To cascade conditions in a join operation, the & logical operation can be used:
emp.join(
    dptm,
    how="left",
    on=(emp.department_id==dptm.department_id) & ((emp.department_id == "101") | (emp.department_id == "102"))
).show()

In [ ]:
# To filter out the records with null department_ids, the isNotNull() can be used:
from pyspark.sql.functions import col

emp.join(
    dptm,
    how="left",
    on=(emp.department_id==dptm.department_id) & ((emp.department_id == "101") | (emp.department_id == "102"))
).filter(dptm.department_id.isNotNull()).show() #Alternatively, the isNull() function can be used.